Loading everything

In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import scipy.sparse as sp
import numpy as np
from codecs import open
import networkx as nx
import community
import random
from time import time

In [11]:
DATA_FOLDER = '/home/teven/fake_news/Wikidumps/'
RAW_FOLDER = os.path.join(DATA_FOLDER, 'raw/')
LINKS_FILE = os.path.join(DATA_FOLDER, '2110_symmetrized_links.json')
FINAL_INDEX_FILE = os.path.join(DATA_FOLDER, '2110_final_index.json')
ADJACENCY_MATRIX = os.path.join(DATA_FOLDER, '2110_conflict_adjacency.txt.npz')
MATRIX_INDEX = os.path.join(DATA_FOLDER, '2110_conflict_matrix_index.txt')
TALK_DATA_FILE = os.path.join(DATA_FOLDER, '2110_talk_data.json')
CONTROVERSY_SCORES = os.path.join(DATA_FOLDER, '2110_controversy_scores.txt')
REVERSE_INDEX_FILE = os.path.join(DATA_FOLDER, '2110_reverse_index.json')
EDIT_WARS = os.path.join(DATA_FOLDER, 'edit_wars.json')

Controversy metric clustering

In [3]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    return sp.csr_matrix(([max(0, value - 0.1) for value in loader['data']], loader['indices'], loader['indptr']), shape=loader['shape'])

In [5]:
adjacency_matrix = load_sparse_csr(ADJACENCY_MATRIX)
adjacency_matrix.eliminate_zeros()
print adjacency_matrix.shape
print len(adjacency_matrix.data)
g = nx.from_scipy_sparse_matrix(adjacency_matrix)
index = json.load(open(MATRIX_INDEX, 'r', encoding='utf-8'), encoding='utf8')
clustering = community.best_partition(g)

(33287, 33287)
2630905


In [6]:
reverse_index = json.load(open(REVERSE_INDEX_FILE, 'r', encoding='utf8'), encoding='utf8')
matrix_index = json.load(open(MATRIX_INDEX, 'r', encoding='utf8'), encoding='utf8')

In [7]:
cluster_range = set(clustering.values())
clusters = [[] for _ in cluster_range]
for article, cluster_number in clustering.iteritems():
    clusters[cluster_number].append(reverse_index[str(matrix_index[article])])
clusters.sort(key=len, reverse=True)

In [8]:
i = 205
print len(clusters[i])
print clusters[i]

25
[u'Deforestation', u'Lolita', u'Japanese sword', u'Tang dynasty', u"Rosie O'Donnell", u'Baa, Baa, Black Sheep', u'Women and children first', u'Nicky Morgan', u'Chloroform', u'Ninjat\u014d', u'Superstition', u'Upstate New York', u'Peter Scott (educationalist)', u'ChucK', u'Primeval', u'Marxism', u'Teddington', u'Kathie Lee Gifford', u'WALL-E', u'Zac Goldsmith', u'Differential heat treatment', u'Kingston University', u'A-League transfers for 2015\u201316 season', u'Tom Holland (actor)', u'Masonic lodge officers']


In [9]:
nx.set_node_attributes(g, {node: random.random() for node in g.nodes()}, "controversy")
nx.set_node_attributes(g, {node: random.randint(0, 9) for node in g.nodes()}, "cluster")
nx.set_node_attributes(g, {node: reverse_index[str(matrix_index[node])] for node in g.nodes()}, "title")

nx.write_gpickle(g, "sample_graph.pickle")

Edit wars clustering

In [12]:
edit_wars = json.load(open(EDIT_WARS))
print 'done'
wars_index = json.load(open(FINAL_INDEX_FILE))
print 'done'
wars_index = {war: wars_index.get(war, None) for war in edit_wars}
print 'done'
wars_index = {k: v for k, v in wars_index.iteritems() if v is not None}
print 'done'
wars_links = json.load(open(LINKS_FILE))
print 'done'
wars_links = {war: wars_links.get(str(index), None) for war, index in wars_index.iteritems()}
print 'done'
wars_links = {k: v for k, v in wars_links.iteritems() if v is not None}
print 'done'
edit_wars = {k: v for k, v in edit_wars.iteritems() if k in wars_index}

done
done
done
done
done
done
done


In [13]:
g = nx.Graph()
g.add_nodes_from(sorted(wars_index.values()))
print len(g.nodes())
print len(wars_index)
nx.set_node_attributes(g, name="title", values={v: k for k, v in wars_index.iteritems()})
nx.set_node_attributes(g, name="controversy_value", values={wars_index[k]: v for k, v in edit_wars.iteritems()})

12539
13043


In [14]:
all_edges = []
start_time = time()
nodes_set = set(wars_index.values())
for i, node in enumerate(g.nodes(data=True)):
    try:
        title = node[1]['title']
    except:
        print node
    edges = [(node[0], child) for child in wars_links[title] if child in nodes_set]
    all_edges.extend(edges)
    if i % 10 == 0:
        print i
        print time() - start_time

g.add_edges_from(all_edges)

0
0.00700616836548
10
0.0117321014404
20
0.0133891105652
30
0.0168631076813
40
0.0239081382751
50
0.0303611755371
60
0.033762216568
70
0.0384271144867
80
0.0393650531769
90
0.041374206543
100
0.0425310134888
110
0.0428872108459
120
0.0448071956635
130
0.0485651493073
140
0.0491681098938
150
0.0498700141907
160
0.0503010749817
170
0.051164150238
180
0.0520770549774
190
0.0525510311127
200
0.0533201694489
210
0.0536091327667
220
0.0542280673981
230
0.0547580718994
240
0.0550971031189
250
0.0559060573578
260
0.0568661689758
270
0.05717420578
280
0.0580351352692
290
0.0591161251068
300
0.0599091053009
310
0.0611231327057
320
0.0616710186005
330
0.0621020793915
340
0.0631282329559
350
0.0644361972809
360
0.0648350715637
370
0.0654411315918
380
0.0667881965637
390
0.0672721862793
400
0.0708529949188
410
0.0711150169373
420
0.0714001655579
430
0.0717551708221
440
0.0722160339355
450
0.0725181102753
460
0.073352098465
470
0.0750710964203
480
0.0755701065063
490
0.075923204422
500
0.07704520225

4760
0.62472820282
4770
0.62704706192
4780
0.627796173096
4790
0.628379106522
4800
0.628708124161
4810
0.629562139511
4820
0.630173206329
4830
0.630931138992
4840
0.638527154922
4850
0.645000219345
4860
0.646857023239
4870
0.648247003555
4880
0.648461103439
4890
0.64945602417
4900
0.655060052872
4910
0.65905714035
4920
0.659426212311
4930
0.659895181656
4940
0.660541057587
4950
0.661900997162
4960
0.66242313385
4970
0.662628173828
4980
0.666102170944
4990
0.667110204697
5000
0.668664216995
5010
0.66960310936
5020
0.669968128204
5030
0.670440196991
5040
0.670739173889
5050
0.67284822464
5060
0.675423145294
5070
0.675930023193
5080
0.677287101746
5090
0.67985200882
5100
0.680855035782
5110
0.68172121048
5120
0.682057142258
5130
0.682792186737
5140
0.683225154877
5150
0.68398809433
5160
0.684710025787
5170
0.685178041458
5180
0.685600996017
5190
0.686993122101
5200
0.68735909462
5210
0.688291072845
5220
0.689038038254
5230
0.68999004364
5240
0.692301034927
5250
0.693090200424
5260
0.69402

1.30252122879
11330
1.30401515961
11340
1.30464506149
11350
1.30490112305
11360
1.30651712418
11370
1.30710601807
11380
1.31124401093
11390
1.31314611435
11400
1.3136510849
11410
1.31429409981
11420
1.31510806084
11430
1.32014203072
11440
1.32082605362
11450
1.3217151165
11460
1.32259202003
11470
1.32317113876
11480
1.32343411446
11490
1.32416510582
11500
1.3251080513
11510
1.32553720474
11520
1.33844208717
11530
1.33871412277
11540
1.33932900429
11550
1.33982515335
11560
1.3400812149
11570
1.34052109718
11580
1.34089422226
11590
1.34320402145
11600
1.34368109703
11610
1.34393906593
11620
1.34444403648
11630
1.34763216972
11640
1.34804821014
11650
1.34856700897
11660
1.34896206856
11670
1.34921503067
11680
1.34957814217
11690
1.34991812706
11700
1.35057210922
11710
1.35088419914
11720
1.35128116608
11730
1.35287117958
11740
1.35335421562
11750
1.35552120209
11760
1.35591506958
11770
1.3562412262
11780
1.35971403122
11790
1.36031508446
11800
1.36077308655
11810
1.36286115646
11820
1.363

In [15]:
g.size()

133192

In [16]:
clustering = community.best_partition(g)
print len(set(clustering.values()))
nx.set_node_attributes(g, name='cluster', values=clustering)

612


In [17]:
title_clusters = [set() for _ in range(len(set(clustering.values())))]
node_clusters = [set() for _ in range(len(set(clustering.values())))]
for i, node in enumerate(g.nodes(data=True)):
    title = node[1]['title']
    cluster = node[1]['cluster']
    title_clusters[cluster].add(title)
    node_clusters[cluster].add(node[0])
title_clusters.sort(key=len)
node_clusters.sort(key=len)

In [18]:
h = g.subgraph([node for node in node_clusters[-1]])

h.size()

smaller_clustering = community.best_partition(h)
print len(set(smaller_clustering.values()))

smaller_title_clusters = [set() for _ in range(len(set(smaller_clustering.values())))]
smaller_node_clusters = [set() for _ in range(len(set(smaller_clustering.values())))]
for i, node in enumerate(h.nodes(data=True)):
    title = node[1]['title']
    cluster = smaller_clustering[node[0]]
    smaller_title_clusters[cluster].add(title)
    smaller_node_clusters[cluster].add(node[0])
smaller_title_clusters.sort(key=len)
smaller_node_clusters.sort(key=len)

7


In [19]:
smaller_title_clusters[-1]
len(smaller_title_clusters[-1])

964

In [20]:
i = g.subgraph([node for node in smaller_node_clusters[-1]])

i.size()

even_smaller_clustering = community.best_partition(i)
print len(set(even_smaller_clustering.values()))

even_smaller_title_clusters = [set() for _ in range(len(set(even_smaller_clustering.values())))]
even_smaller_node_clusters = [set() for _ in range(len(set(even_smaller_clustering.values())))]
for i, node in enumerate(i.nodes(data=True)):
    title = node[1]['title']
    cluster = even_smaller_clustering[node[0]]
    even_smaller_title_clusters[cluster].add(title)
    even_smaller_node_clusters[cluster].add(node[0])
even_smaller_title_clusters.sort(key=len)
even_smaller_node_clusters.sort(key=len)

9


In [26]:
even_smaller_title_clusters[-4]

{u'14 On Fire',
 u'2008\u20132012 Icelandic financial crisis',
 u'2012 Nobel Peace Prize',
 u'2015 Thalys attack',
 u'Accession of the Republic of Macedonia to the European Union',
 u'Afrikaner',
 u'Alternative for Germany',
 u'Antoine Gounet',
 u'Arms Industry',
 u'Attention-deficit hyperactivity disorder',
 u'Austin Macauley Publishers',
 u'Belgian cuisine',
 u'Belgium',
 u'Bichon Fris\xe9',
 u'Boat',
 u'Bose stereo speakers',
 u'Bulgarian parliamentary election, 2013',
 u'Charles Feltman',
 u'Child pornography',
 u'Ciro Immobile',
 u'Coat of arms of Luxembourg',
 u'Coleshill, Warwickshire',
 u'Council of the European Union',
 u'Darwinius',
 u'Demographics of atheism',
 u'Dennis Ketcham',
 u'Developed country',
 u'Digital Audio Broadcasting',
 u'Djembe',
 u'Doctor of Philosophy',
 u'Domain of Soissons',
 u'Double degree',
 u'Duchy of Aquitaine',
 u'Dutch Empire',
 u'Dutch Gold',
 u'Dutch language',
 u'Dutch people',
 u'Earthenware',
 u'Edition Peters',
 u'Electric fence',
 u'Emich F\

In [22]:
final_node_clusters = node_clusters[:-1] + smaller_node_clusters[:-1] + even_smaller_node_clusters
final_title_clusters = title_clusters[:-1] + smaller_title_clusters[:-1] + even_smaller_title_clusters
final_node_clusters.sort(key=len, reverse=True)
final_title_clusters.sort(key=len, reverse=True)

In [23]:
final_clustering = {}
for i, node_cluster in enumerate(final_node_clusters):
    for node in node_cluster:
        final_clustering[node] = i
nx.set_node_attributes(g, name='cluster', values=final_clustering)

In [ ]:
nx.write_gpickle(g, "final_graph.pickle")

In [ ]:
graph_dict = {"graph": [],
             "links": [],
             "nodes": [],
             "directed": False,
             "multigraph": False}

Dump of the graph visualization

In [17]:
def cluster_score(cluster_number):
    total_number = len(final_node_clusters)
    return ((cluster_number * 107) % total_number) /float(total_number)

node_index = {node: i for i, node in enumerate(g.nodes())}
indexed_edges = [{"source": node_index[edge[0]], "target": node_index[edge[1]]} for edge in g.edges]
indexed_nodes = [{"size": node[1]["controversy_value"], "score": cluster_score(node[1]["cluster"]), "type": "circle", "id": node[1]["title"]} for node in g.nodes(data=True)]

graph_dict["links"] = indexed_edges
graph_dict["nodes"] = indexed_nodes

json.dump(graph_dict, open("d3graph.json", "w"), indent=2)
print len(final_node_clusters)

final_title_clusters[450]